In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import xlsxwriter

# Load the dataset from Excel file
excel_file = 'sam9.xlsx'
sheet1 = pd.read_excel(excel_file, sheet_name='Sheet_1')
sheet2 = pd.read_excel(excel_file, sheet_name='Sheet_2')

# Combine the sheets if necessary
data = pd.concat([sheet1, sheet2], ignore_index=True)

# Parse dates and set the index
data['order_date'] = pd.to_datetime(data['order_date'])
data.set_index('order_date', inplace=True)

# Aggregate total quantity sold per day
daily_data = data.resample('D').sum().fillna(0)

# Split the data into training and testing sets
train = daily_data[daily_data.index.year < daily_data.index.year.max()]
test = daily_data[daily_data.index.year == daily_data.index.year.max()]

# Fit the ETS model on the training data
model = ExponentialSmoothing(train['quantity'], trend='add', seasonal='add', seasonal_periods=365).fit()

# Forecast the quantity for the test period
forecast = model.forecast(len(test))

# Create a DataFrame for the results
results = pd.DataFrame({
    'date': test.index,
    'actual': test['quantity'],
    'forecast': forecast
}).set_index('date')

# Calculate the metrics
mse = np.mean((results['actual'] - results['forecast']) ** 2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(results['actual'] - results['forecast']))
mape = np.mean(np.abs((results['actual'] - results['forecast']) / results['actual'])) * 100
accuracy = 100 - mape

# Print the metrics
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Accuracy: {accuracy:.2f}%")

# Save the results to an Excel file
with pd.ExcelWriter('predictions_qt3.xlsx', engine='xlsxwriter') as writer:
    # Write the results to the Excel file
    results.to_excel(writer, sheet_name='Predictions')
    
    # Create and save the plot
    fig, ax = plt.subplots(figsize=(12, 8))
    results.plot(ax=ax)
    plt.title('Actual vs Predicted Quantities')
    plt.xlabel('Date')
    plt.ylabel('Quantity')
    plt.legend(['Actual', 'Forecast'], loc='upper left')
    plt.tight_layout()

    # Save the plot as an image
    plt.savefig('forecast_plot.png')
    plt.close()

    # Insert the plot into the Excel file
    worksheet = writer.sheets['Predictions']
    worksheet.insert_image('H2', 'forecast_plot.png')

print("Forecasting complete. Results saved to predictions_qt3.xlsx.")
